In [1]:
import os
import torch
import numpy as np
from pesq import pesq
import torch.nn as nn
from pystoi import stoi
from scipy.signal import get_window
import numpy as np
import matplotlib.pyplot as plt
from models import DCCRN
import librosa

--------------------  C  O  N  F  I  G  ----------------------
--------------------------------------------------------------
MODEL INFO : DCCRN
LOSS INFO : SI-SNR, perceptual : False
LSTM : complex
SKIP : True
MASKING INFO : E

BATCH : 10
LEARNING RATE : 0.001
--------------------------------------------------------------
--------------------------------------------------------------



In [2]:
model = [DCCRN().cuda(),DCCRN().cuda(),DCCRN().cuda()]



In [3]:
chkpt_path = "/work/hyerim/remixIT/DNN-based-Speech-Enhancement-in-the-frequency-domain/models/test/SI-SNR_NONE.pt"
checkpoint = torch.load(chkpt_path)
model[0].load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [4]:
chkpt_path = "/work/hyerim/remixIT/DNN-based-Speech-Enhancement-in-the-frequency-domain/models/test/SI-SNR_LMS.pt"
checkpoint = torch.load(chkpt_path)
model[1].load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [5]:
chkpt_path = "/work/hyerim/remixIT/DNN-based-Speech-Enhancement-in-the-frequency-domain/models/test/MSE_PMSQE.pt"
checkpoint = torch.load(chkpt_path)
model[2].load_state_dict(checkpoint['model'])

<All keys matched successfully>

### Tools for Score

In [10]:
###############################################################################
#                           PESQ (another ref)                                #
###############################################################################
# interface to PESQ evaluation, taking in two waveforms as input
def cal_pesq(fs, dirty_wavs, clean_wavs):
    scores = []
    for i in range(len(dirty_wavs)):
        try:
            pesq_score = pesq(fs, dirty_wavs[i], clean_wavs[i], 'nb')
            scores.append(pesq_score)
        except Exception as e:
           
            continue
    return scores




### Test

In [11]:

def preprocess_wav(file_path):
    y, sr = librosa.load(file_path, sr=8000)
    return y

In [12]:

batch_num = 0

avg_pesq_score = 0


In [13]:
fs = 8000

folder_path = '/work/hyerim/remixIT/DNN-based-Speech-Enhancement-in-the-frequency-domain/data/wav/test'
clean_files = [os.path.join(folder_path, f)
               for f in os.listdir(folder_path) if f.endswith('.wav')]




for db in ["-5db", "0db"]:

    batch_num = 0

    folder_path = f"/work/hyerim/remixIT/DNN-based-Speech-Enhancement-in-the-frequency-domain/data/noisy/{db}/test"
    noisy_files = [os.path.join(folder_path, f)
                   for f in os.listdir(folder_path) if f.endswith('.wav')]

    
    
    save_score = [0 for _ in  clean_files]
    pickup_files = []
    
    for idx_model in [0,2]:
        model[idx_model].eval()
        batch_num = 0
        avg_pesq_score = 0

        with torch.no_grad():
            for index, clean_file in enumerate(clean_files):

                batch_num += 1

                noisy_file = noisy_files[index]

                if os.path.basename(clean_file) != os.path.basename(noisy_file):
                    print(f"Not matched {clean_file} {noisy_file}")
                    continue

                inputs, _ = librosa.load(noisy_file, sr=16000)
                targets, _ = librosa.load(clean_file, sr=16000)

                inputs = torch.Tensor(inputs).unsqueeze(dim=0).cuda()
                targets = torch.Tensor(targets).unsqueeze(dim=0).cuda()

                _,_,outputs = model[idx_model](inputs,targets)
                
            

                # estimate the output speech with pesq and stoi
                estimated_wavs = outputs.cpu().detach().numpy()
                clean_wavs = targets.cpu().detach().numpy()
             
                pesq_score = cal_pesq(fs, estimated_wavs, clean_wavs)
                
               
                
                # for i in range(len(pesq_score)):
                #     print(f"PESQ {pesq_score[i]:.6f}")
                

                
                # reshape for sum
                
                pesq_score = np.reshape(pesq_score, (1, -1))
                pesq_score = sum(pesq_score[0]) / len(inputs)
               
               
                
                if idx_model == 0:
                    save_score[index] = pesq_score
                    
                if idx_model == 2:
                    
                    if save_score[index] != 0:
                        ratio = (pesq_score - save_score[index])/save_score[index]
                        
                       
                        
                        if db == '-5db':
                            if ratio > 0.07:
                                pickup_files.append(clean_file)
                        else:
                            if ratio > 0.13:
                                pickup_files.append(clean_file)
                        
               
                avg_pesq_score += pesq_score

       

            avg_pesq_score /= batch_num
            

            print(f"{db} {idx_model} avg_pesq {avg_pesq_score} {len(pickup_files)}")

-5db 0 avg_pesq 3.781325807152214 0
-5db 2 avg_pesq 3.8211362718225836 2
0db 0 avg_pesq 3.9228338485235694 0
0db 2 avg_pesq 3.950789683991736 0
